# crystal_functions tutorial

First, functions need to be imported. Since crystal_functions was installed via pip, python already knows where to find it. Therefore, all functions can be imported following this logic:

```console
from crystal_functions.module_name import function_name
```
In this first exercise, we need to use the Crystal_input class that sits in the file_readwrite module:

In [8]:
from crystal_functions.file_readwrite import Crystal_input

### Build the input from blocks

The first exercise will be creating and writing a CRYSTAL input file. We will see later a more convenient way to do this, but for now we are just going to pass crystal_functions a list of parameters that we need to build the input and then write it. This corresponds to manually writing to file, but instead of writing to file we are creating python lists (delimited by square brackets and using commas as separators):

In [21]:
geom_block = ['CRYSTAL\n',
             '0 0 0\n',
             '225\n',
             '4.217\n',
             '2\n',
             '12 0.    0.    0.\n',
             '8 0.5   0.5   0.5\n',
             'END\n']
bs_block   = ['BASISSET\n','POB-DZVP\n']
func_block = ['DFT\n', 'B3LYP\n', 'XXLGRID\n', 'ENDDFT\n']
scf_block  = [['TOLINTEG\n', '7 7 7 7 14\n'],
             ['SHRINK\n', '12 24\n'],
             ['MAXCYCLE\n', '200\n'],
             ['FMIXING\n', '70\n'],
             'DIIS\n',
             'ENDSCF\n']
title = 'MgO crystal_functions tutorial' #optional

Then we need to create the Crystal_input object. This is called initialising the object:

In [11]:
mgo_input = Crystal_input()

If we try to see what mgo_input, by executing the following command:

In [12]:
print(mgo_input)

You can see this is an object of the class file_readwrite.Crystal_input.

Now that the object has been initialised, we want to add information to it and that is when the lists we prepared earlier will be used. We are going to use the from_file function, which belongs to the Crystal_input class. If you are not familiar with python, you only need to know that if a function belongs to a class, it is called by adding .function_name() after the object created by the class. In this instance, mgo_input is an object of the Crystal_input class. Therefore, to use a function on it we will use:

```console
mgo_input.from_blocks()
```
We are first going to execute the command above and see the error message because it is very informative. 


In [14]:
mgo_input.from_blocks()

TypeError: from_blocks() missing 4 required positional arguments: 'geom_block', 'bs_block', 'func_block', and 'scf_block'

This can be used everytime we are not sure about the arguments for a function. You should see that the from_blocks() takes four positional arguments. Positional, in this context means mandatory. There is also a fifth optional one, which is “title”. The reason why this is optional is because a default value has already been defined in the function itself.

Now we can run the command 

In [22]:
mgo_input.from_blocks(geom_block=geom_block, bs_block=bs_block, 
                                        func_block=func_block, scf_block=scf_block, title=title)

If we didn’t include title, crystal_functions would use “crystal_functions generated input” as first line of the input.

Now mgo_input will contain the information about the different blocks. We can test this by executing:

In [23]:
mgo_input.geom_block

['CRYSTAL\n',
 '0 0 0\n',
 '225\n',
 '4.217\n',
 '2\n',
 '12 0.    0.    0.\n',
 '8 0.5   0.5   0.5\n',
 'END\n']

This displays the geom_block. 

You can also visualise the whole input by using:

In [24]:
mgo_input.print_input()

MgO crystal_functions tutorial
CRYSTAL
0 0 0
225
4.217
2
12 0.    0.    0.
8 0.5   0.5   0.5
END
BASISSET
POB-DZVP
DFT
B3LYP
XXLGRID
ENDDFT
TOLINTEG
7 7 7 7 14
SHRINK
12 24
MAXCYCLE
200
FMIXING
70
DIIS
ENDSCF


### Set the runcry and runprop path (optional)

# REMOVE BELOW

In [10]:
import sys
sys.path.insert(1, '../crystal_functions/')
from file_readwrite import Crystal_input